Extracción de los datos de los premios Oscar con Beautiful Soup:
- Fecha de la ceremonia
- Mejor película
- Mejor director.
- Mejor actor.
- Mejor actriz.

In [38]:
# Instalacion Librerias
# antes de empezar importamos las librerías que vamos a usar. 
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [39]:
def sacar_oscar(url):
    res_oscar = requests.get(url) # hacemos la request a la página de la que queremos sacar la info
    if res_oscar.status_code == 200: # # Verificar si la solicitud fue exitosa (código de estado 200)
        sopa_oscar = BeautifulSoup(res_oscar.content, 'html.parser') ## creamos el objeto BeautifulSoup para poder acceder al contenido solicitado

        tabla_oscar = sopa_oscar.find('table', {'class': 'wikitable'}) # Encontrar la tabla que contiene la información de los premios Oscar
        
        diccionario = {"numero": [], # Crear un diccionario para almacenar los datos de las películas
                       "ceremonia": [],   
                       "pelicula": [], 
                       "director": [], 
                       "actor": [], 
                       "actriz": []}
        filas = tabla_oscar.findAll("tr")[1:] # Encontrar todas las filas de la tabla excepto la primera (que generalmente contiene los encabezados)
        
        for fila in filas: ## Iterar sobre cada fila de la tabla y extraer los datos
            celdas = fila.findAll("td")
            diccionario["numero"].append(celdas[0].text.strip())
            diccionario["ceremonia"].append(re.search(r'\d{4}', celdas[1].text.strip()).group() if re.search(r'\d{4}', celdas[1].text.strip()) else celdas[1].text.strip()) #expresión regular para buscar un patrón de cuatro dígitos consecutivos en el texto de la ceremonia.
            diccionario["pelicula"].append(celdas[2].text.strip())
            diccionario["director"].append(celdas[3].text.strip())
            diccionario["actor"].append(celdas[4].text.strip())
            diccionario["actriz"].append(celdas[5].text.strip())
        
        df_peliculas = pd.DataFrame(diccionario)
        
        return df_peliculas
    else:
        print("Error al realizar la solicitud. Código de estado:", res_oscar.status_code)
        return None

url_oscar = "https://es.wikipedia.org/wiki/Premios_Óscar" # URL de la página de Wikipedia que contiene la información de los premios Oscar
df_premios_oscar = sacar_oscar(url_oscar) # Llamar a la función y obtener el DataFrame con los datos de las películas de los premios Oscar

print(df_premios_oscar) #Imprimir el DataFrame


   numero ceremonia                                           pelicula  \
0      1°      1929                                Alas (Wings) (1927)   
1      2°      1930  La melodía de Broadway (The Broadway melody) (...   
2      3°      1930  Sin novedad en el frente (All quiet on the wes...   
3      4°      1931                                    Cimarron (1931)   
4      5°      1932                                        Grand Hotel   
..    ...       ...                                                ...   
91    92°      2020                                           Parasite   
92    93°      2021                                          Nomadland   
93    94°      2022                                               CODA   
94    95°      2023                  Everything Everywhere All at Once   
95    95°      2024                                        Oppenheimer   

                                             director  \
0               Drama:  F. BorzageCom.:  L. Milestone 

In [40]:
df = sacar_oscar("https://es.wikipedia.org/wiki/Premios_%C3%93scar")
df.head()

,numero,ceremonia,pelicula,director,actor,actriz
0,1°,1929,Alas (Wings) (1927),Drama: F. BorzageCom.: L. Milestone,E. Jannings(The Last Command),J. Gaynor(7th Heaven)
1,2°,1930,La melodía de Broadway (The Broadway melody) (...,F. Lloyd(The Divine Lady),W. Baxter(In Old Arizona),M. Pickford(Coquette)
2,3°,1930,Sin novedad en el frente (All quiet on the wes...,L. MilestoneSin novedad en el frente (All quie...,G. Arliss(Disraeli),N. Shearer(The Divorcee)
3,4°,1931,Cimarron (1931),N. Taurog(Skippy),L. Barrymore(A Free Soul),M. Dressler(Min and Bill)
4,5°,1932,Grand Hotel,F. Borzage(Bad Girl),F. March W. Beery,H. Hayes(The Sin of Madelon Claudet)


In [41]:
def filtrar_por_anio(df):
    
    df['ceremonia'] = pd.to_datetime(df['ceremonia']) # Convertir la columna "ceremonia" a tipo datetime
    
    
    df_filtrado = df[(df['ceremonia'].dt.year >= 2015) & (df['ceremonia'].dt.year <= 2020)] # Filtrar las películas que están entre los años 2015 y 2020
    
    return df_filtrado




In [42]:
df_premios_2015_2020 = filtrar_por_anio(df_premios_oscar) # Aplicar la función para filtrar el DataFrame por el campo "ceremonia" entre 2015 y 2020


print(df_premios_2015_2020) # Imprimir el DataFrame filtrado


   numero  ceremonia                                         pelicula  \
86    87° 2015-01-01  Birdman or (The Unexpected Virtue of Ignorance)   
87    88° 2016-01-01                                        Spotlight   
88    89° 2017-01-01                                        Moonlight   
89    90° 2018-01-01                               The Shape of Water   
90    91° 2019-01-01                                       Green Book   
91    92° 2020-01-01                                         Parasite   

                                             director  \
86  A. G. Iñárritu(Birdman or (The Unexpected Virt...   
87                       A. G. Iñárritu(The Revenant)   
88                            D. Chazelle(La La Land)   
89                    G. del Toro(The Shape of Water)   
90                                    A. Cuarón(Roma)   
91                                  Bong J.(Parasite)   

                                    actor  \
86  E. Redmayne(The Theory Of Everything)  

In [43]:
df_premios_2015_2020.head()


,numero,ceremonia,pelicula,director,actor,actriz
86,87°,2015-01-01,Birdman or (The Unexpected Virtue of Ignorance),A. G. Iñárritu(Birdman or (The Unexpected Virt...,E. Redmayne(The Theory Of Everything),J.Moore(Still Alice)
87,88°,2016-01-01,Spotlight,A. G. Iñárritu(The Revenant),L. DiCaprio(The Revenant),B. Larson(Room)
88,89°,2017-01-01,Moonlight,D. Chazelle(La La Land),C. Affleck(Manchester by the Sea),E. Stone(La La Land)
89,90°,2018-01-01,The Shape of Water,G. del Toro(The Shape of Water),G. Oldman(Darkest Hour),"F. McDormand(Three Billboards Outside Ebbing, ..."
90,91°,2019-01-01,Green Book,A. Cuarón(Roma),R. Malek(Bohemian Rhapsody),O. Colman(La favorita)


In [44]:
df_premios_2015_2020.tail()

,numero,ceremonia,pelicula,director,actor,actriz
87,88°,2016-01-01,Spotlight,A. G. Iñárritu(The Revenant),L. DiCaprio(The Revenant),B. Larson(Room)
88,89°,2017-01-01,Moonlight,D. Chazelle(La La Land),C. Affleck(Manchester by the Sea),E. Stone(La La Land)
89,90°,2018-01-01,The Shape of Water,G. del Toro(The Shape of Water),G. Oldman(Darkest Hour),"F. McDormand(Three Billboards Outside Ebbing, ..."
90,91°,2019-01-01,Green Book,A. Cuarón(Roma),R. Malek(Bohemian Rhapsody),O. Colman(La favorita)
91,92°,2020-01-01,Parasite,Bong J.(Parasite),J. Phoenix(Joker),R. Zellweger(Judy)


In [45]:
def convertir_df_a_lista_tuplas(df): #Creamos una funcion para convertir la lista a tuplas
    lista_tuplas = []
    for fila in df.values: #iteramos por el df de premios del 2015 al 2020
        
        ano_ceremonia = fila[1].year # Extraer el año de la fecha de ceremonia y usamos year para coger solo el año
        
        tupla = (ano_ceremonia, fila[2], fila[3], fila[4], fila[5]) # Crear la tupla con el formato deseado
        lista_tuplas.append(tupla) # Agregamos la tupla recién creada a la lista de tuplas utilizando el método append()
    return lista_tuplas # Lista de tuplas resultante


lista_tuplas_premios_2015_2020 = convertir_df_a_lista_tuplas(df_premios_2015_2020) # Almacenar el DataFrame filtrado en una lista de tuplas


print(lista_tuplas_premios_2015_2020) # Imprimir la lista de tuplas


[(2015, 'Birdman or (The Unexpected Virtue of Ignorance)', 'A. G. Iñárritu(Birdman or (The Unexpected Virtue of Ignorance))', 'E. Redmayne(The Theory Of Everything)', 'J.Moore(Still Alice)'), (2016, 'Spotlight', 'A. G. Iñárritu(The Revenant)', 'L. DiCaprio(The Revenant)', 'B. Larson(Room)'), (2017, 'Moonlight', 'D. Chazelle(La La Land)', 'C. Affleck(Manchester by the Sea)', 'E. Stone(La La Land)'), (2018, 'The Shape of Water', 'G. del Toro(The Shape of Water)', 'G. Oldman(Darkest Hour)', 'F. McDormand(Three Billboards Outside Ebbing, Missouri)'), (2019, 'Green Book', 'A. Cuarón(Roma)', 'R. Malek(Bohemian Rhapsody)', 'O. Colman(La favorita)'), (2020, 'Parasite', 'Bong J.(Parasite)', 'J. Phoenix(Joker)', 'R. Zellweger(Judy)')]
